# train.py(face_pytorch)

In [1]:
import torchvision.transforms as transforms
import torch.utils.data as data
import numpy as np
import cv2
import os
import torch
from torch.nn import DataParallel
from datetime import datetime
import time
import numpy as np
import torchvision.transforms as transforms
import argparse
from torch.optim import lr_scheduler
import torch.optim as optim
from easydict import EasyDict
from torch import nn
import math
from torch import nn
from torch.nn import Parameter
import torch.nn.functional as F
import visdom
import time
import logging
import torchvision.transforms as transforms
import torch.utils.data as data
import scipy.io
import json
import torch.utils.data
import torchvision.transforms as transforms
from torch.nn import DataParallel




# backbone

## mobilefacenet.py

In [2]:
# from backbone.mobilefacenet import MobileFaceNet
MobileFaceNet_BottleNeck_Setting = [
    # t, c , n ,s
    [2, 64, 5, 2],
    [4, 128, 1, 2],
    [2, 128, 6, 1],
    [4, 128, 1, 2],
    [2, 128, 2, 1]
]

class BottleNeck(nn.Module):
    def __init__(self, inp, oup, stride, expansion):
        super(BottleNeck, self).__init__()
        self.connect = stride == 1 and inp == oup

        self.conv = nn.Sequential(
            # 1*1 conv
            nn.Conv2d(inp, inp * expansion, 1, 1, 0, bias=False),
            nn.BatchNorm2d(inp * expansion),
            nn.PReLU(inp * expansion),

            # 3*3 depth wise conv
            nn.Conv2d(inp * expansion, inp * expansion, 3, stride, 1, groups=inp * expansion, bias=False),
            nn.BatchNorm2d(inp * expansion),
            nn.PReLU(inp * expansion),

            # 1*1 conv
            nn.Conv2d(inp * expansion, oup, 1, 1, 0, bias=False),
            nn.BatchNorm2d(oup),
        )

    def forward(self, x):
        if self.connect:
            return x + self.conv(x)
        else:
            return self.conv(x)


class ConvBlock(nn.Module):
    def __init__(self, inp, oup, k, s, p, dw=False, linear=False):
        super(ConvBlock, self).__init__()
        self.linear = linear
        if dw:
            self.conv = nn.Conv2d(inp, oup, k, s, p, groups=inp, bias=False)
        else:
            self.conv = nn.Conv2d(inp, oup, k, s, p, bias=False)

        self.bn = nn.BatchNorm2d(oup)
        if not linear:
            self.prelu = nn.PReLU(oup)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        if self.linear:
            return x
        else:
            return self.prelu(x)


class MobileFaceNet(nn.Module):
    def __init__(self, feature_dim=128, bottleneck_setting=MobileFaceNet_BottleNeck_Setting):
        super(MobileFaceNet, self).__init__()
        self.conv1 = ConvBlock(3, 64, 3, 2, 1)
        self.dw_conv1 = ConvBlock(64, 64, 3, 1, 1, dw=True)

        self.cur_channel = 64
        block = BottleNeck
        self.blocks = self._make_layer(block, bottleneck_setting)

        self.conv2 = ConvBlock(128, 512, 1, 1, 0)
        self.linear7 = ConvBlock(512, 512, 7, 1, 0, dw=True, linear=True)
        self.linear1 = ConvBlock(512, feature_dim, 1, 1, 0, linear=True)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, setting):
        layers = []
        for t, c, n, s in setting:
            for i in range(n):
                if i == 0:
                    layers.append(block(self.cur_channel, c, s, t))
                else:
                    layers.append(block(self.cur_channel, c, 1, t))
                self.cur_channel = c

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.dw_conv1(x)
        x = self.blocks(x)
        x = self.conv2(x)
        x = self.linear7(x)
        x = self.linear1(x)
        x = x.view(x.size(0), -1)

        return x


if __name__ == "__main__":
    input = torch.Tensor(2, 3, 112, 112)
    net = MobileFaceNet()
    print(net)

    x = net(input)
    print(x.shape)

MobileFaceNet(
  (conv1): ConvBlock(
    (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (prelu): PReLU(num_parameters=64)
  )
  (dw_conv1): ConvBlock(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (prelu): PReLU(num_parameters=64)
  )
  (blocks): Sequential(
    (0): BottleNeck(
      (conv): Sequential(
        (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): PReLU(num_parameters=128)
        (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=128, bias=False)
        (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

torch.Size([2, 128])


## cbam.py

In [3]:
# from backbone.cbam import CBAMResNet
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

class SEModule(nn.Module):
    '''Squeeze and Excitation Module'''
    def __init__(self, channels, reduction):
        super(SEModule, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Conv2d(channels, channels // reduction, kernel_size=1, padding=0, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Conv2d(channels // reduction, channels, kernel_size=1, padding=0, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        input = x
        x = self.avg_pool(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)

        return input * x

class CAModule(nn.Module):
    '''Channel Attention Module'''
    def __init__(self, channels, reduction):
        super(CAModule, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.shared_mlp = nn.Sequential(nn.Conv2d(channels, channels // reduction, kernel_size=1, padding=0, bias=False),
                                        nn.ReLU(inplace=True),
                                        nn.Conv2d(channels // reduction, channels, kernel_size=1, padding=0, bias=False))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        input = x
        avg_pool = self.avg_pool(x)
        max_pool = self.max_pool(x)
        x = self.shared_mlp(avg_pool) + self.shared_mlp(max_pool)
        x = self.sigmoid(x)

        return input * x

class SAModule(nn.Module):
    '''Spatial Attention Module'''
    def __init__(self):
        super(SAModule, self).__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size=3, padding=1, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        input = x
        avg_c = torch.mean(x, 1, True)
        max_c, _ = torch.max(x, 1, True)
        x = torch.cat((avg_c, max_c), 1)
        x = self.conv(x)
        x = self.sigmoid(x)
        return input * x

class BottleNeck_IR(nn.Module):
    '''Improved Residual Bottlenecks'''
    def __init__(self, in_channel, out_channel, stride, dim_match):
        super(BottleNeck_IR, self).__init__()
        self.res_layer = nn.Sequential(nn.BatchNorm2d(in_channel),
                                       nn.Conv2d(in_channel, out_channel, (3, 3), 1, 1, bias=False),
                                       nn.BatchNorm2d(out_channel),
                                       nn.PReLU(out_channel),
                                       nn.Conv2d(out_channel, out_channel, (3, 3), stride, 1, bias=False),
                                       nn.BatchNorm2d(out_channel))
        if dim_match:
            self.shortcut_layer = None
        else:
            self.shortcut_layer = nn.Sequential(
                nn.Conv2d(in_channel, out_channel, kernel_size=(1, 1), stride=stride, bias=False),
                nn.BatchNorm2d(out_channel)
            )

    def forward(self, x):
        shortcut = x
        res = self.res_layer(x)

        if self.shortcut_layer is not None:
            shortcut = self.shortcut_layer(x)

        return shortcut + res

class BottleNeck_IR_SE(nn.Module):
    '''Improved Residual Bottlenecks with Squeeze and Excitation Module'''
    def __init__(self, in_channel, out_channel, stride, dim_match):
        super(BottleNeck_IR_SE, self).__init__()
        self.res_layer = nn.Sequential(nn.BatchNorm2d(in_channel),
                                       nn.Conv2d(in_channel, out_channel, (3, 3), 1, 1, bias=False),
                                       nn.BatchNorm2d(out_channel),
                                       nn.PReLU(out_channel),
                                       nn.Conv2d(out_channel, out_channel, (3, 3), stride, 1, bias=False),
                                       nn.BatchNorm2d(out_channel),
                                       SEModule(out_channel, 16))
        if dim_match:
            self.shortcut_layer = None
        else:
            self.shortcut_layer = nn.Sequential(
                nn.Conv2d(in_channel, out_channel, kernel_size=(1, 1), stride=stride, bias=False),
                nn.BatchNorm2d(out_channel)
            )

    def forward(self, x):
        shortcut = x
        res = self.res_layer(x)

        if self.shortcut_layer is not None:
            shortcut = self.shortcut_layer(x)

        return shortcut + res

class BottleNeck_IR_CAM(nn.Module):
    '''Improved Residual Bottlenecks with Channel Attention Module'''
    def __init__(self, in_channel, out_channel, stride, dim_match):
        super(BottleNeck_IR_CAM, self).__init__()
        self.res_layer = nn.Sequential(nn.BatchNorm2d(in_channel),
                                       nn.Conv2d(in_channel, out_channel, (3, 3), 1, 1, bias=False),
                                       nn.BatchNorm2d(out_channel),
                                       nn.PReLU(out_channel),
                                       nn.Conv2d(out_channel, out_channel, (3, 3), stride, 1, bias=False),
                                       nn.BatchNorm2d(out_channel),
                                       CAModule(out_channel, 16))
        if dim_match:
            self.shortcut_layer = None
        else:
            self.shortcut_layer = nn.Sequential(
                nn.Conv2d(in_channel, out_channel, kernel_size=(1, 1), stride=stride, bias=False),
                nn.BatchNorm2d(out_channel)
            )

    def forward(self, x):
        shortcut = x
        res = self.res_layer(x)

        if self.shortcut_layer is not None:
            shortcut = self.shortcut_layer(x)

        return shortcut + res

class BottleNeck_IR_SAM(nn.Module):
    '''Improved Residual Bottlenecks with Spatial Attention Module'''
    def __init__(self, in_channel, out_channel, stride, dim_match):
        super(BottleNeck_IR_SAM, self).__init__()
        self.res_layer = nn.Sequential(nn.BatchNorm2d(in_channel),
                                       nn.Conv2d(in_channel, out_channel, (3, 3), 1, 1, bias=False),
                                       nn.BatchNorm2d(out_channel),
                                       nn.PReLU(out_channel),
                                       nn.Conv2d(out_channel, out_channel, (3, 3), stride, 1, bias=False),
                                       nn.BatchNorm2d(out_channel),
                                       SAModule())
        if dim_match:
            self.shortcut_layer = None
        else:
            self.shortcut_layer = nn.Sequential(
                nn.Conv2d(in_channel, out_channel, kernel_size=(1, 1), stride=stride, bias=False),
                nn.BatchNorm2d(out_channel)
            )

    def forward(self, x):
        shortcut = x
        res = self.res_layer(x)

        if self.shortcut_layer is not None:
            shortcut = self.shortcut_layer(x)

        return shortcut + res

class BottleNeck_IR_CBAM(nn.Module):
    '''Improved Residual Bottleneck with Channel Attention Module and Spatial Attention Module'''
    def __init__(self, in_channel, out_channel, stride, dim_match):
        super(BottleNeck_IR_CBAM, self).__init__()
        self.res_layer = nn.Sequential(nn.BatchNorm2d(in_channel),
                                       nn.Conv2d(in_channel, out_channel, (3, 3), 1, 1, bias=False),
                                       nn.BatchNorm2d(out_channel),
                                       nn.PReLU(out_channel),
                                       nn.Conv2d(out_channel, out_channel, (3, 3), stride, 1, bias=False),
                                       nn.BatchNorm2d(out_channel),
                                       CAModule(out_channel, 16),
                                       SAModule()
                                       )
        if dim_match:
            self.shortcut_layer = None
        else:
            self.shortcut_layer = nn.Sequential(
                nn.Conv2d(in_channel, out_channel, kernel_size=(1, 1), stride=stride, bias=False),
                nn.BatchNorm2d(out_channel)
            )

    def forward(self, x):
        shortcut = x
        res = self.res_layer(x)

        if self.shortcut_layer is not None:
            shortcut = self.shortcut_layer(x)

        return shortcut + res


filter_list = [64, 64, 128, 256, 512]
def get_layers(num_layers):
    if num_layers == 50:
        return [3, 4, 14, 3]
    elif num_layers == 100:
        return [3, 13, 30, 3]
    elif num_layers == 152:
        return [3, 8, 36, 3]

class CBAMResNet(nn.Module):
    def __init__(self, num_layers, feature_dim=512, drop_ratio=0.4, mode='ir',filter_list=filter_list):
        super(CBAMResNet, self).__init__()
        assert num_layers in [50, 100, 152], 'num_layers should be 50, 100 or 152'
        assert mode in ['ir', 'ir_se', 'ir_cam', 'ir_sam', 'ir_cbam'], 'mode should be ir, ir_se, ir_cam, ir_sam or ir_cbam'
        layers = get_layers(num_layers)
        if mode == 'ir':
            block = BottleNeck_IR
        elif mode == 'ir_se':
            block = BottleNeck_IR_SE
        elif mode == 'ir_cam':
            block = BottleNeck_IR_CAM
        elif mode == 'ir_sam':
            block = BottleNeck_IR_SAM
        elif mode == 'ir_cbam':
            block = BottleNeck_IR_CBAM

        self.input_layer = nn.Sequential(nn.Conv2d(3, 64, (3, 3), stride=1, padding=1, bias=False),
                                         nn.BatchNorm2d(64),
                                         nn.PReLU(64))
        self.layer1 = self._make_layer(block, filter_list[0], filter_list[1], layers[0], stride=2)
        self.layer2 = self._make_layer(block, filter_list[1], filter_list[2], layers[1], stride=2)
        self.layer3 = self._make_layer(block, filter_list[2], filter_list[3], layers[2], stride=2)
        self.layer4 = self._make_layer(block, filter_list[3], filter_list[4], layers[3], stride=2)

        self.output_layer = nn.Sequential(nn.BatchNorm2d(512),
                                          nn.Dropout(drop_ratio),
                                          Flatten(),
                                          nn.Linear(512 * 7 * 7, feature_dim),
                                          nn.BatchNorm1d(feature_dim))

        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0.0)
            elif isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, block, in_channel, out_channel, blocks, stride):
        layers = []
        layers.append(block(in_channel, out_channel, stride, False))
        for i in range(1, blocks):
            layers.append(block(out_channel, out_channel, 1, True))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.input_layer(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.output_layer(x)

        return x

## attention.py

In [4]:
# from backbone.attention import ResidualAttentionNet_56, ResidualAttentionNet_92
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

class ResidualBlock(nn.Module):

    def __init__(self, in_channel, out_channel, stride=1):
        super(ResidualBlock, self).__init__()
        self.in_channel = in_channel
        self.out_channel = out_channel
        self.stride = stride

        self.res_bottleneck = nn.Sequential(nn.BatchNorm2d(in_channel),
                                            nn.ReLU(inplace=True),
                                            nn.Conv2d(in_channel, out_channel//4, 1, 1, bias=False),
                                            nn.BatchNorm2d(out_channel//4),
                                            nn.ReLU(inplace=True),
                                            nn.Conv2d(out_channel//4, out_channel//4, 3, stride, padding=1, bias=False),
                                            nn.BatchNorm2d(out_channel//4),
                                            nn.ReLU(inplace=True),
                                            nn.Conv2d(out_channel//4, out_channel, 1, 1, bias=False))
        self.shortcut = nn.Conv2d(in_channel, out_channel, 1, stride, bias=False)

    def forward(self, x):
        res = x
        out = self.res_bottleneck(x)
        if self.in_channel != self.out_channel or self.stride != 1:
            res = self.shortcut(x)

        out += res
        return out

class AttentionModule_stage1(nn.Module):

    # input size is 56*56
    def __init__(self, in_channel, out_channel, size1=(56, 56), size2=(28, 28), size3=(14, 14)):
        super(AttentionModule_stage1, self).__init__()
        self.share_residual_block = ResidualBlock(in_channel, out_channel)
        self.trunk_branches = nn.Sequential(ResidualBlock(in_channel, out_channel),
                                            ResidualBlock(in_channel, out_channel))

        self.mpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.mask_block1 = ResidualBlock(in_channel, out_channel)
        self.skip_connect1 = ResidualBlock(in_channel, out_channel)

        self.mpool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.mask_block2 = ResidualBlock(in_channel, out_channel)
        self.skip_connect2 = ResidualBlock(in_channel, out_channel)

        self.mpool3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.mask_block3 = nn.Sequential(ResidualBlock(in_channel, out_channel),
                                         ResidualBlock(in_channel, out_channel))

        self.interpolation3 = nn.UpsamplingBilinear2d(size=size3)
        self.mask_block4 = ResidualBlock(in_channel, out_channel)

        self.interpolation2 = nn.UpsamplingBilinear2d(size=size2)
        self.mask_block5 = ResidualBlock(in_channel, out_channel)

        self.interpolation1 = nn.UpsamplingBilinear2d(size=size1)
        self.mask_block6 = nn.Sequential(nn.BatchNorm2d(out_channel),
                                         nn.ReLU(inplace=True),
                                         nn.Conv2d(out_channel, out_channel, 1, 1, bias=False),
                                         nn.BatchNorm2d(out_channel),
                                         nn.ReLU(inplace=True),
                                         nn.Conv2d(out_channel, out_channel, 1, 1, bias=False),
                                         nn.Sigmoid())

        self.last_block = ResidualBlock(in_channel, out_channel)

    def forward(self, x):
        x = self.share_residual_block(x)
        out_trunk = self.trunk_branches(x)

        out_pool1 = self.mpool1(x)
        out_block1 = self.mask_block1(out_pool1)
        out_skip_connect1 = self.skip_connect1(out_block1)

        out_pool2 = self.mpool2(out_block1)
        out_block2 = self.mask_block2(out_pool2)
        out_skip_connect2 = self.skip_connect2(out_block2)

        out_pool3 = self.mpool3(out_block2)
        out_block3 = self.mask_block3(out_pool3)
        #
        out_inter3 = self.interpolation3(out_block3) + out_block2
        out = out_inter3 + out_skip_connect2
        out_block4 = self.mask_block4(out)

        out_inter2 = self.interpolation2(out_block4) + out_block1
        out = out_inter2 + out_skip_connect1
        out_block5 = self.mask_block5(out)

        out_inter1 = self.interpolation1(out_block5) + out_trunk
        out_block6 = self.mask_block6(out_inter1)

        out = (1 + out_block6) + out_trunk
        out_last = self.last_block(out)

        return out_last

class AttentionModule_stage2(nn.Module):

    # input image size is 28*28
    def __init__(self, in_channels, out_channels, size1=(28, 28), size2=(14, 14)):
        super(AttentionModule_stage2, self).__init__()
        self.first_residual_blocks = ResidualBlock(in_channels, out_channels)

        self.trunk_branches = nn.Sequential(
            ResidualBlock(in_channels, out_channels),
            ResidualBlock(in_channels, out_channels)
         )

        self.mpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.softmax1_blocks = ResidualBlock(in_channels, out_channels)
        self.skip1_connection_residual_block = ResidualBlock(in_channels, out_channels)

        self.mpool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.softmax2_blocks = nn.Sequential(
            ResidualBlock(in_channels, out_channels),
            ResidualBlock(in_channels, out_channels)
        )

        self.interpolation2 = nn.UpsamplingBilinear2d(size=size2)
        self.softmax3_blocks = ResidualBlock(in_channels, out_channels)
        self.interpolation1 = nn.UpsamplingBilinear2d(size=size1)
        self.softmax4_blocks = nn.Sequential(
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=1, stride=1, bias=False),
            nn.Sigmoid()
        )
        self.last_blocks = ResidualBlock(in_channels, out_channels)

    def forward(self, x):
        x = self.first_residual_blocks(x)
        out_trunk = self.trunk_branches(x)
        out_mpool1 = self.mpool1(x)
        out_softmax1 = self.softmax1_blocks(out_mpool1)
        out_skip1_connection = self.skip1_connection_residual_block(out_softmax1)

        out_mpool2 = self.mpool2(out_softmax1)
        out_softmax2 = self.softmax2_blocks(out_mpool2)

        out_interp2 = self.interpolation2(out_softmax2) + out_softmax1
        out = out_interp2 + out_skip1_connection

        out_softmax3 = self.softmax3_blocks(out)
        out_interp1 = self.interpolation1(out_softmax3) + out_trunk
        out_softmax4 = self.softmax4_blocks(out_interp1)
        out = (1 + out_softmax4) * out_trunk
        out_last = self.last_blocks(out)

        return out_last

class AttentionModule_stage3(nn.Module):

    # input image size is 14*14
    def __init__(self, in_channels, out_channels, size1=(14, 14)):
        super(AttentionModule_stage3, self).__init__()
        self.first_residual_blocks = ResidualBlock(in_channels, out_channels)

        self.trunk_branches = nn.Sequential(
            ResidualBlock(in_channels, out_channels),
            ResidualBlock(in_channels, out_channels)
         )

        self.mpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.softmax1_blocks = nn.Sequential(
            ResidualBlock(in_channels, out_channels),
            ResidualBlock(in_channels, out_channels)
        )

        self.interpolation1 = nn.UpsamplingBilinear2d(size=size1)

        self.softmax2_blocks = nn.Sequential(
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=1, stride=1, bias=False),
            nn.Sigmoid()
        )

        self.last_blocks = ResidualBlock(in_channels, out_channels)

    def forward(self, x):
        x = self.first_residual_blocks(x)
        out_trunk = self.trunk_branches(x)
        out_mpool1 = self.mpool1(x)
        out_softmax1 = self.softmax1_blocks(out_mpool1)

        out_interp1 = self.interpolation1(out_softmax1) + out_trunk
        out_softmax2 = self.softmax2_blocks(out_interp1)
        out = (1 + out_softmax2) * out_trunk
        out_last = self.last_blocks(out)

        return out_last

class ResidualAttentionNet_56(nn.Module):

    # for input size 112
    def __init__(self, feature_dim=512, drop_ratio=0.4):
        super(ResidualAttentionNet_56, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )
        self.mpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.residual_block1 = ResidualBlock(64, 256)
        self.attention_module1 = AttentionModule_stage1(256, 256)
        self.residual_block2 = ResidualBlock(256, 512, 2)
        self.attention_module2 = AttentionModule_stage2(512, 512)
        self.residual_block3 = ResidualBlock(512, 512, 2)
        self.attention_module3 = AttentionModule_stage3(512, 512)
        self.residual_block4 = ResidualBlock(512, 512, 2)
        self.residual_block5 = ResidualBlock(512, 512)
        self.residual_block6 = ResidualBlock(512, 512)
        self.output_layer = nn.Sequential(nn.BatchNorm2d(512),
                                          nn.Dropout(drop_ratio),
                                          Flatten(),
                                          nn.Linear(512 * 7 * 7, feature_dim),
                                          nn.BatchNorm1d(feature_dim))

    def forward(self, x):
        out = self.conv1(x)
        out = self.mpool1(out)
        # print(out.data)
        out = self.residual_block1(out)
        out = self.attention_module1(out)
        out = self.residual_block2(out)
        out = self.attention_module2(out)
        out = self.residual_block3(out)
        # print(out.data)
        out = self.attention_module3(out)
        out = self.residual_block4(out)
        out = self.residual_block5(out)
        out = self.residual_block6(out)
        out = self.output_layer(out)

        return out

class ResidualAttentionNet_92(nn.Module):

    # for input size 112
    def __init__(self, feature_dim=512, drop_ratio=0.4):
        super(ResidualAttentionNet_92, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )
        self.mpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.residual_block1 = ResidualBlock(64, 256)
        self.attention_module1 = AttentionModule_stage1(256, 256)
        self.residual_block2 = ResidualBlock(256, 512, 2)
        self.attention_module2 = AttentionModule_stage2(512, 512)
        self.attention_module2_2 = AttentionModule_stage2(512, 512)  # tbq add
        self.residual_block3 = ResidualBlock(512, 1024, 2)
        self.attention_module3 = AttentionModule_stage3(1024, 1024)
        self.attention_module3_2 = AttentionModule_stage3(1024, 1024)  # tbq add
        self.attention_module3_3 = AttentionModule_stage3(1024, 1024)  # tbq add
        self.residual_block4 = ResidualBlock(1024, 2048, 2)
        self.residual_block5 = ResidualBlock(2048, 2048)
        self.residual_block6 = ResidualBlock(2048, 2048)
        self.output_layer = nn.Sequential(nn.BatchNorm2d(2048),
                                          nn.Dropout(drop_ratio),
                                          Flatten(),
                                          nn.Linear(2048 * 7 * 7, feature_dim),
                                          nn.BatchNorm1d(feature_dim))

    def forward(self, x):
        out = self.conv1(x)
        out = self.mpool1(out)
        # print(out.data)
        out = self.residual_block1(out)
        out = self.attention_module1(out)
        out = self.residual_block2(out)
        out = self.attention_module2(out)
        out = self.attention_module2_2(out)
        out = self.residual_block3(out)
        # print(out.data)
        out = self.attention_module3(out)
        out = self.attention_module3_2(out)
        out = self.attention_module3_3(out)
        out = self.residual_block4(out)
        out = self.residual_block5(out)
        out = self.residual_block6(out)
        out = self.output_layer(out)

        return out

# margin

## ArcMarginProduct.py

In [5]:
# from margin.ArcMarginProduct import ArcMarginProduct
class ArcMarginProduct(nn.Module):
    def __init__(self, in_feature=128, out_feature=10575, s=32.0, m=0.50, easy_margin=False):
        super(ArcMarginProduct, self).__init__()
        self.in_feature = in_feature
        self.out_feature = out_feature
        self.s = s
        self.m = m
        self.weight = Parameter(torch.Tensor(out_feature, in_feature))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)

        # make the function cos(theta+m) monotonic decreasing while theta in [0°,180°]
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, x, label):
        # cos(theta)
        cosine = F.linear(F.normalize(x), F.normalize(self.weight))
        # cos(theta + m)
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m

        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where((cosine - self.th) > 0, phi, cosine - self.mm)

        #one_hot = torch.zeros(cosine.size(), device='cuda' if torch.cuda.is_available() else 'cpu')
        one_hot = torch.zeros_like(cosine)
        one_hot.scatter_(1, label.view(-1, 1), 1)
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output = output * self.s

        return output

## MultiMarginProduct.py

In [6]:
# from margin.MultiMarginProduct import MultiMarginProduct
class MultiMarginProduct(nn.Module):
    def __init__(self, in_feature=128, out_feature=10575, s=32.0, m1=0.20, m2=0.35, easy_margin=False):
        super(MultiMarginProduct, self).__init__()
        self.in_feature = in_feature
        self.out_feature = out_feature
        self.s = s
        self.m1 = m1
        self.m2 = m2
        self.weight = Parameter(torch.Tensor(out_feature, in_feature))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m1 = math.cos(m1)
        self.sin_m1 = math.sin(m1)

        # make the function cos(theta+m) monotonic decreasing while theta in [0°,180°]
        self.th = math.cos(math.pi - m1)
        self.mm = math.sin(math.pi - m1) * m1

    def forward(self, x, label):
        # cos(theta)
        cosine = F.linear(F.normalize(x), F.normalize(self.weight))
        # cos(theta + m1)
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m1 - sine * self.sin_m1

        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where((cosine - self.th) > 0, phi, cosine - self.mm)


        one_hot = torch.zeros_like(cosine)
        one_hot.scatter_(1, label.view(-1, 1), 1)
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine) # additive angular margin
        output = output - one_hot * self.m2 # additive cosine margin
        output = output * self.s

        return output


## CosineMarginProduct.py

In [7]:
# from margin.CosineMarginProduct import CosineMarginProduct
class CosineMarginProduct(nn.Module):
    def __init__(self, in_feature=128, out_feature=10575, s=30.0, m=0.35):
        super(CosineMarginProduct, self).__init__()
        self.in_feature = in_feature
        self.out_feature = out_feature
        self.s = s
        self.m = m
        self.weight = Parameter(torch.Tensor(out_feature, in_feature))
        nn.init.xavier_uniform_(self.weight)


    def forward(self, input, label):
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        # one_hot = torch.zeros(cosine.size(), device='cuda' if torch.cuda.is_available() else 'cpu')
        one_hot = torch.zeros_like(cosine)
        one_hot.scatter_(1, label.view(-1, 1), 1.0)

        output = self.s * (cosine - one_hot * self.m)
        return output

## InnerProduct.py

In [8]:
# from margin.InnerProduct import InnerProduct
class InnerProduct(nn.Module):
    def __init__(self, in_feature=128, out_feature=10575):
        super(InnerProduct, self).__init__()
        self.in_feature = in_feature
        self.out_feature = out_feature

        self.weight = Parameter(torch.Tensor(out_feature, in_feature))
        nn.init.xavier_uniform_(self.weight)


    def forward(self, input, label):
        # label not used
        output = F.linear(input, self.weight)
        return output


# Utils

## Visualize.py

In [9]:
# from utils.visualize import Visualizer
class Visualizer():
    def __init__(self, env='default', **kwargs):
        self.vis = visdom.Visdom(env=env, **kwargs)
        self.index = 1

    def plot_curves(self, d, iters, title='loss', xlabel='iters', ylabel='accuracy'):
        name = list(d.keys())
        val = list(d.values())
        if len(val) == 1:
            y = np.array(val)
        else:
            y = np.array(val).reshape(-1, len(val))
        self.vis.line(Y=y,
                      X=np.array([self.index]),
                      win=title,
                      opts=dict(legend=name, title = title, xlabel=xlabel, ylabel=ylabel),
                      update=None if self.index == 0 else 'append')
        self.index = iters


if __name__ == '__main__':
    vis = Visualizer(env='test')
    for i in range(10):
        x = i
        y = 2 * i
        z = 4 * i
        vis.plot_curves({'train': x, 'test': y}, iters=i, title='train')
        vis.plot_curves({'train': z, 'test': y, 'val': i}, iters=i, title='test')
        time.sleep(1)

Setting up a new session...


Exception in user code:
------------------------------------------------------------


Traceback (most recent call last):
  File "c:\users\shiney\documents\workspace\pytorch\lib\site-packages\urllib3\connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "c:\users\shiney\documents\workspace\pytorch\lib\site-packages\urllib3\util\connection.py", line 83, in create_connection
    raise err
  File "c:\users\shiney\documents\workspace\pytorch\lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\shiney\documents\workspace\pytorch\lib\site-packages\urllib3\connectionpool.py", line 601, in urlopen
    chunked=chunked)
  File "c:\users\shiney\documents\workspace\pytorch\lib\site-packages\urllib3\connectionpool.py", line 357, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File 

Exception in user code:
------------------------------------------------------------


Traceback (most recent call last):
  File "c:\users\shiney\documents\workspace\pytorch\lib\site-packages\urllib3\connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "c:\users\shiney\documents\workspace\pytorch\lib\site-packages\urllib3\util\connection.py", line 83, in create_connection
    raise err
  File "c:\users\shiney\documents\workspace\pytorch\lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\shiney\documents\workspace\pytorch\lib\site-packages\urllib3\connectionpool.py", line 601, in urlopen
    chunked=chunked)
  File "c:\users\shiney\documents\workspace\pytorch\lib\site-packages\urllib3\connectionpool.py", line 357, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File 

...

Exception in user code:
------------------------------------------------------------


Traceback (most recent call last):
  File "c:\users\shiney\documents\workspace\pytorch\lib\site-packages\urllib3\connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "c:\users\shiney\documents\workspace\pytorch\lib\site-packages\urllib3\util\connection.py", line 83, in create_connection
    raise err
  File "c:\users\shiney\documents\workspace\pytorch\lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\shiney\documents\workspace\pytorch\lib\site-packages\urllib3\connectionpool.py", line 601, in urlopen
    chunked=chunked)
  File "c:\users\shiney\documents\workspace\pytorch\lib\site-packages\urllib3\connectionpool.py", line 357, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File 

Exception in user code:
------------------------------------------------------------


Traceback (most recent call last):
  File "c:\users\shiney\documents\workspace\pytorch\lib\site-packages\urllib3\connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "c:\users\shiney\documents\workspace\pytorch\lib\site-packages\urllib3\util\connection.py", line 83, in create_connection
    raise err
  File "c:\users\shiney\documents\workspace\pytorch\lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\shiney\documents\workspace\pytorch\lib\site-packages\urllib3\connectionpool.py", line 601, in urlopen
    chunked=chunked)
  File "c:\users\shiney\documents\workspace\pytorch\lib\site-packages\urllib3\connectionpool.py", line 357, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File 

KeyboardInterrupt: 

## Ligging.py

In [13]:
# from utils.logging import init_log
def init_log(output_dir):
    logging.basicConfig(level=logging.DEBUG,
                        format='%(asctime)s %(message)s',
                        datefmt='%Y%m%d-%H:%M:%S',
                        filename=os.path.join(output_dir, 'log.log'),
                        filemode='w')
    console = logging.StreamHandler()
    console.setLevel(logging.INFO)
    logging.getLogger('').addHandler(console)
    return logging


# dataset
## casia_sebface.py

In [14]:
# from dataset.casia_webface import CASIAWebFace
def img_loader(path):
    try:
        with open(path, 'rb') as f:
            img = cv2.imread(path)
            if len(img.shape) == 2:
                img = np.stack([img] * 3, 2)
            return img
    except IOError:
        print('Cannot load image ' + path)


class CASIAWebFace(data.Dataset):
    def __init__(self, root, file_list, transform=None, loader=img_loader):

        self.root = root
        self.transform = transform
        self.loader = loader

        image_list = []
        label_list = []
        with open(file_list) as f:
            img_label_list = f.read().splitlines()
        for info in img_label_list:
            image_path, label_name = info.split(' ')
            image_list.append(image_path)
            label_list.append(int(label_name))

        self.image_list = image_list
        self.label_list = label_list
        self.class_nums = len(np.unique(self.label_list))
        print("dataset size: ", len(self.image_list), '/', self.class_nums)

    def __getitem__(self, index):
        img_path = self.image_list[index]
        label = self.label_list[index]

        img = self.loader(os.path.join(self.root, img_path))

        # random flip with ratio of 0.5
        flip = np.random.choice(2) * 2 - 1
        if flip == 1:
            img = cv2.flip(img, 1)

        if self.transform is not None:
            img = self.transform(img)
        else:
            img = torch.from_numpy(img)

        return img, label

    def __len__(self):
        return len(self.image_list)

## lfw.py

In [15]:
# from dataset.lfw import LFW
def img_loader(path):
    try:
        with open(path, 'rb') as f:
            img = cv2.imread(path)
            if len(img.shape) == 2:
                img = np.stack([img] * 3, 2)
            return img
    except IOError:
        print('Cannot load image ' + path)

class LFW(data.Dataset):
    def __init__(self, root, file_list, transform=None, loader=img_loader):

        self.root = root
        self.file_list = file_list
        self.transform = transform
        self.loader = loader
        self.nameLs = []
        self.nameRs = []
        self.folds = []
        self.flags = []

        with open(file_list) as f:
            pairs = f.read().splitlines()[1:]
        for i, p in enumerate(pairs):
            p = p.split('\t')
            if len(p) == 3:
                nameL = p[0] + '/' + p[0] + '_' + '{:04}.jpg'.format(int(p[1]))
                nameR = p[0] + '/' + p[0] + '_' + '{:04}.jpg'.format(int(p[2]))
                fold = i // 600
                flag = 1
            elif len(p) == 4:
                nameL = p[0] + '/' + p[0] + '_' + '{:04}.jpg'.format(int(p[1]))
                nameR = p[2] + '/' + p[2] + '_' + '{:04}.jpg'.format(int(p[3]))
                fold = i // 600
                flag = -1
            self.nameLs.append(nameL)
            self.nameRs.append(nameR)
            self.folds.append(fold)
            self.flags.append(flag)

    def __getitem__(self, index):

        img_l = self.loader(os.path.join(self.root, self.nameLs[index]))
        img_r = self.loader(os.path.join(self.root, self.nameRs[index]))
        imglist = [img_l, cv2.flip(img_l, 1), img_r, cv2.flip(img_r, 1)]

        if self.transform is not None:
            for i in range(len(imglist)):
                imglist[i] = self.transform(imglist[i])

            imgs = imglist
            return imgs
        else:
            imgs = [torch.from_numpy(i) for i in imglist]
            return imgs

    def __len__(self):
        return len(self.nameLs)


## agedb.py

In [16]:
# from dataset.agedb import AgeDB30
def img_loader(path):
    try:
        with open(path, 'rb') as f:
            img = cv2.imread(path)
            if len(img.shape) == 2:
                img = np.stack([img] * 3, 2)
            return img
    except IOError:
        print('Cannot load image ' + path)

class AgeDB30(data.Dataset):
    def __init__(self, root, file_list, transform=None, loader=img_loader):

        self.root = root
        self.file_list = file_list
        self.transform = transform
        self.loader = loader
        self.nameLs = []
        self.nameRs = []
        self.folds = []
        self.flags = []

        with open(file_list) as f:
            pairs = f.read().splitlines()
        for i, p in enumerate(pairs):
            p = p.split(' ')
            nameL = p[0]
            nameR = p[1]
            fold = i // 600
            flag = int(p[2])

            self.nameLs.append(nameL)
            self.nameRs.append(nameR)
            self.folds.append(fold)
            self.flags.append(flag)

    def __getitem__(self, index):

        img_l = self.loader(os.path.join(self.root, self.nameLs[index]))
        img_r = self.loader(os.path.join(self.root, self.nameRs[index]))
        imglist = [img_l, cv2.flip(img_l, 1), img_r, cv2.flip(img_r, 1)]

        if self.transform is not None:
            for i in range(len(imglist)):
                imglist[i] = self.transform(imglist[i])

            imgs = imglist
            return imgs
        else:
            imgs = [torch.from_numpy(i) for i in imglist]
            return imgs

    def __len__(self):
        return len(self.nameLs)

## cfp.py

In [17]:
# from dataset.cfp import CFP_FP
def img_loader(path):
    try:
        with open(path, 'rb') as f:
            img = cv2.imread(path)
            if len(img.shape) == 2:
                img = np.stack([img] * 3, 2)
            return img
    except IOError:
        print('Cannot load image ' + path)

class CFP_FP(data.Dataset):
    def __init__(self, root, file_list, transform=None, loader=img_loader):

        self.root = root
        self.file_list = file_list
        self.transform = transform
        self.loader = loader
        self.nameLs = []
        self.nameRs = []
        self.folds = []
        self.flags = []

        with open(file_list) as f:
            pairs = f.read().splitlines()
        for i, p in enumerate(pairs):
            p = p.split(' ')
            nameL = p[0]
            nameR = p[1]
            fold = i // 700
            flag = int(p[2])

            self.nameLs.append(nameL)
            self.nameRs.append(nameR)
            self.folds.append(fold)
            self.flags.append(flag)

    def __getitem__(self, index):

        img_l = self.loader(os.path.join(self.root, self.nameLs[index]))
        img_r = self.loader(os.path.join(self.root, self.nameRs[index]))
        imglist = [img_l, cv2.flip(img_l, 1), img_r, cv2.flip(img_r, 1)]

        if self.transform is not None:
            for i in range(len(imglist)):
                imglist[i] = self.transform(imglist[i])

            imgs = imglist
            return imgs
        else:
            imgs = [torch.from_numpy(i) for i in imglist]
            return imgs

    def __len__(self):
        return len(self.nameLs)


# eval_lfw.py

In [18]:
# from eval_lfw import evaluation_10_fold, getFeatureFromTorch
def getAccuracy(scores, flags, threshold):
    p = np.sum(scores[flags == 1] > threshold)
    n = np.sum(scores[flags == -1] < threshold)
    return 1.0 * (p + n) / len(scores)

def getThreshold(scores, flags, thrNum):
    accuracys = np.zeros((2 * thrNum + 1, 1))
    thresholds = np.arange(-thrNum, thrNum + 1) * 1.0 / thrNum
    for i in range(2 * thrNum + 1):
        accuracys[i] = getAccuracy(scores, flags, thresholds[i])
    max_index = np.squeeze(accuracys == np.max(accuracys))
    bestThreshold = np.mean(thresholds[max_index])
    return bestThreshold

def evaluation_10_fold(feature_path='./result/cur_epoch_result.mat'):
    ACCs = np.zeros(10)
    result = scipy.io.loadmat(feature_path)
    for i in range(10):
        fold = result['fold']
        flags = result['flag']
        featureLs = result['fl']
        featureRs = result['fr']

        valFold = fold != i
        testFold = fold == i
        flags = np.squeeze(flags)

        mu = np.mean(np.concatenate((featureLs[valFold[0], :], featureRs[valFold[0], :]), 0), 0)
        mu = np.expand_dims(mu, 0)
        featureLs = featureLs - mu
        featureRs = featureRs - mu
        featureLs = featureLs / np.expand_dims(np.sqrt(np.sum(np.power(featureLs, 2), 1)), 1)
        featureRs = featureRs / np.expand_dims(np.sqrt(np.sum(np.power(featureRs, 2), 1)), 1)

        scores = np.sum(np.multiply(featureLs, featureRs), 1)
        threshold = getThreshold(scores[valFold[0]], flags[valFold[0]], 10000)
        ACCs[i] = getAccuracy(scores[testFold[0]], flags[testFold[0]], threshold)

    return ACCs

def loadModel(data_root, file_list, backbone_net, gpus='0', resume=None):

    if backbone_net == 'MobileFace':
        net = mobilefacenet.MobileFaceNet()
    elif backbone_net == 'CBAM_50':
        net = cbam.CBAMResNet(50, feature_dim=args.feature_dim, mode='ir')
    elif backbone_net == 'CBAM_50_SE':
        net = cbam.CBAMResNet(50, feature_dim=args.feature_dim, mode='ir_se')
    elif backbone_net == 'CBAM_100':
        net = cbam.CBAMResNet(100, feature_dim=args.feature_dim, mode='ir')
    elif backbone_net == 'CBAM_100_SE':
        net = cbam.CBAMResNet(100, feature_dim=args.feature_dim, mode='ir_se')
    else:
        print(backbone_net, ' is not available!')

    # gpu init
    multi_gpus = False
    if len(gpus.split(',')) > 1:
        multi_gpus = True
    os.environ['CUDA_VISIBLE_DEVICES'] = gpus
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    net.load_state_dict(torch.load(resume)['net_state_dict'])

    if multi_gpus:
        net = DataParallel(net).to(device)
    else:
        net = net.to(device)

    transform = transforms.Compose([
        transforms.ToTensor(),  # range [0, 255] -> [0.0,1.0]
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))  # range [0.0, 1.0] -> [-1.0,1.0]
    ])
    lfw_dataset = LFW(data_root, file_list, transform=transform)
    lfw_loader = torch.utils.data.DataLoader(lfw_dataset, batch_size=128,
                                             shuffle=False, num_workers=2, drop_last=False)

    return net.eval(), device, lfw_dataset, lfw_loader

def getFeatureFromTorch(feature_save_dir, net, device, data_set, data_loader):
    featureLs = None
    featureRs = None
    count = 0
    for data in data_loader:
        for i in range(len(data)):
            data[i] = data[i].to(device)
        count += data[0].size(0)
        #print('extracing deep features from the face pair {}...'.format(count))
        with torch.no_grad():
            res = [net(d).data.cpu().numpy() for d in data]
        featureL = np.concatenate((res[0], res[1]), 1)
        featureR = np.concatenate((res[2], res[3]), 1)
        # print(featureL.shape, featureR.shape)
        if featureLs is None:
            featureLs = featureL
        else:
            featureLs = np.concatenate((featureLs, featureL), 0)
        if featureRs is None:
            featureRs = featureR
        else:
            featureRs = np.concatenate((featureRs, featureR), 0)
        # print(featureLs.shape, featureRs.shape)

    result = {'fl': featureLs, 'fr': featureRs, 'fold': data_set.folds, 'flag': data_set.flags}
    scipy.io.savemat(feature_save_dir, result)


In [19]:
def img_loader(path):
    try:
        with open(path, 'rb') as f:
            img = cv2.imread(path)
            if len(img.shape) == 2:
                img = np.stack([img] * 3, 2)
            return img
    except IOError:
        print('Cannot load image ' + path)

In [20]:
transform = transforms.Compose([
    transforms.ToTensor(),  # range [0, 255] -> [0.0,1.0]
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))  # range [0.0, 1.0] -> [-1.0,1.0]
])

In [21]:
class CASIAWebFace(data.Dataset):
    def __init__(self, root, file_list, transform=None, loader=img_loader):

        self.root = root
        self.transform = transform
        self.loader = loader

        image_list = []
        label_list = []
        with open(file_list) as f:
            img_label_list = f.read().splitlines()
        for info in img_label_list:
            image_path, label_name = info.split('\t')[1:]
            image_list.append(image_path)
            label_list.append(int(label_name))

        self.image_list = image_list
        self.label_list = label_list
        self.class_nums = len(np.unique(self.label_list))
        print("dataset size: ", len(self.image_list), '/', self.class_nums)

    def __getitem__(self, index):
        img_path = self.image_list[index]
        label = self.label_list[index]

        img = self.loader(os.path.join(self.root, img_path))

        # random flip with ratio of 0.5
        flip = np.random.choice(2) * 2 - 1
        if flip == 1:
            img = cv2.flip(img, 1)

        if self.transform is not None:
            img = self.transform(img)
        else:
            img = torch.from_numpy(img)

        return img, label

    def __len__(self):
        return len(self.image_list)

# train.py

In [27]:
# gpu init
multi_gpus = False
if len(args.gpus.split(',')) > 1:
    multi_gpus = True
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpus
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [28]:
# log init
save_dir = os.path.join(args.save_dir, args.model_pre + args.backbone.upper() + '_' + datetime.now().strftime('%Y%m%d_%H%M%S'))
if os.path.exists(save_dir):
    raise NameError('model dir exists!')
os.makedirs(save_dir)
logging = init_log(save_dir)
_print = logging.info

In [29]:
# dataset loader
transform = transforms.Compose([
    transforms.ToTensor(),  # range [0, 255] -> [0.0,1.0]
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))  # range [0.0, 1.0] -> [-1.0,1.0]
])
# validation dataset
print(args.train_root)
trainset = CASIAWebFace(args.train_root, args.train_file_list, transform=transform)


dataset size:  590 / 3


In [30]:
trainset.class_nums

3

In [31]:
trainset.image_list

['../data/small_vgg/small_vgg_112x112/train/n000001/0001_01.jpg',
 '../data/small_vgg/small_vgg_112x112/train/n000001/0002_01.jpg',
 '../data/small_vgg/small_vgg_112x112/train/n000001/0003_01.jpg',
 '../data/small_vgg/small_vgg_112x112/train/n000001/0005_01.jpg',
 '../data/small_vgg/small_vgg_112x112/train/n000001/0006_01.jpg',
 '../data/small_vgg/small_vgg_112x112/train/n000001/0007_01.jpg',
 '../data/small_vgg/small_vgg_112x112/train/n000001/0008_01.jpg',
 '../data/small_vgg/small_vgg_112x112/train/n000001/0010_01.jpg',
 '../data/small_vgg/small_vgg_112x112/train/n000001/0011_01.jpg',
 '../data/small_vgg/small_vgg_112x112/train/n000001/0013_01.jpg',
 '../data/small_vgg/small_vgg_112x112/train/n000001/0014_01.jpg',
 '../data/small_vgg/small_vgg_112x112/train/n000001/0015_01.jpg',
 '../data/small_vgg/small_vgg_112x112/train/n000001/0016_01.jpg',
 '../data/small_vgg/small_vgg_112x112/train/n000001/0017_01.jpg',
 '../data/small_vgg/small_vgg_112x112/train/n000001/0018_01.jpg',
 '../data/

In [32]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=args.batch_size,
                                          shuffle=True,  drop_last=False)   # num_workers=8,
print(trainloader)

In [33]:
trainloader.batch_size

32

In [34]:
# test dataset
lfwdataset = LFW(args.lfw_test_root, args.lfw_file_list, transform=transform)
lfwloader = torch.utils.data.DataLoader(lfwdataset, batch_size=128,
                                         shuffle=False, num_workers=4, drop_last=False)
# agedbdataset = AgeDB30(args.agedb_test_root, args.agedb_file_list, transform=transform)
# agedbloader = torch.utils.data.DataLoader(agedbdataset, batch_size=128,
#                                         shuffle=False, num_workers=4, drop_last=False)
# cfpfpdataset = CFP_FP(args.cfpfp_test_root, args.cfpfp_file_list, transform=transform)
# cfpfploader = torch.utils.data.DataLoader(cfpfpdataset, batch_size=128,
#                                           shuffle=False, num_workers=4, drop_last=False)

FileNotFoundError: [Errno 2] No such file or directory: '/media/sda/lfw/pairs.txt'

In [35]:
# define backbone and margin layer
if args.backbone == 'MobileFace':
    net = MobileFaceNet()
elif args.backbone == 'Res50_IR':
    net = CBAMResNet(50, feature_dim=args.feature_dim, mode='ir')
elif args.backbone == 'SERes50_IR':
    net = CBAMResNet(50, feature_dim=args.feature_dim, mode='ir_se')
elif args.backbone == 'Res100_IR':
    net = CBAMResNet(100, feature_dim=args.feature_dim, mode='ir')
elif args.backbone == 'SERes100_IR':
    net = CBAMResNet(100, feature_dim=args.feature_dim, mode='ir_se')
elif args.backbone == 'Attention_56':
    net = ResidualAttentionNet_56(feature_dim=args.feature_dim)
elif args.backbone == 'Attention_92':
    net = ResidualAttentionNet_92(feature_dim=args.feature_dim)
else:
    print(args.backbone, ' is not available!')

if args.margin_type == 'ArcFace':
    margin = ArcMarginProduct(args.feature_dim, trainset.class_nums, s=args.scale_size)
elif args.margin_type == 'MultiMargin':
    margin = MultiMarginProduct(args.feature_dim, trainset.class_nums, s=args.scale_size)
elif args.margin_type == 'CosFace':
    margin = CosineMarginProduct(args.feature_dim, trainset.class_nums, s=args.scale_size)
elif args.margin_type == 'Softmax':
    margin = InnerProduct(args.feature_dim, trainset.class_nums)
elif args.margin_type == 'SphereFace':
    pass
else:
    print(args.margin_type, 'is not available!')

if args.resume:
    print('resume the model parameters from: ', args.net_path, args.margin_path)
    net.load_state_dict(torch.load(args.net_path)['net_state_dict'])
    margin.load_state_dict(torch.load(args.margin_path)['net_state_dict'])



In [36]:
# define optimizers for different layer
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer_ft = optim.SGD([
    {'params': net.parameters(), 'weight_decay': 5e-4},
    {'params': margin.parameters(), 'weight_decay': 5e-4}
], lr=0.1, momentum=0.9, nesterov=True)
exp_lr_scheduler = lr_scheduler.MultiStepLR(optimizer_ft, milestones=[6, 11, 16], gamma=0.1)

if multi_gpus:
    net = DataParallel(net).to(device)
    margin = DataParallel(margin).to(device)
else:
    net = net.to(device)
    margin = margin.to(device)

In [37]:
best_lfw_acc = 0.0
best_lfw_iters = 0
best_agedb30_acc = 0.0
best_agedb30_iters = 0
best_cfp_fp_acc = 0.0
best_cfp_fp_iters = 0
total_iters = 0
# vis = Visualizer(env=args.model_pre + args.backbone)

In [41]:
for epoch in range(1, args.total_epoch + 1):
# while(1):
    exp_lr_scheduler.step()
    # train model
    _print('Train Epoch: {}/{} ...'.format(epoch, args.total_epoch))
    net.train()

    since = time.time()
    for data in trainloader:
        print('len(data)', len(data), data[0].shape, data[1].shape)
        img, label = data[0].to(device), data[1].to(device)
        optimizer_ft.zero_grad()

        raw_logits = net(img)
        output = margin(raw_logits, label)
        total_loss = criterion(output, label)
        total_loss.backward()
        optimizer_ft.step()

        total_iters += 1
        # print train information
        if total_iters % 100 == 0:
            # current training accuracy
            _, predict = torch.max(output.data, 1)
            total = label.size(0)
            correct = (np.array(predict.cpu()) == np.array(label.data.cpu())).sum()
            time_cur = (time.time() - since) / 100
            since = time.time()
#             vis.plot_curves({'softmax loss': total_loss.item()}, iters=total_iters, title='train loss',
#                             xlabel='iters', ylabel='train loss')
#             vis.plot_curves({'train accuracy': correct / total}, iters=total_iters, title='train accuracy', xlabel='iters',
#                             ylabel='train accuracy')

            _print("Iters: {:0>6d}/[{:0>2d}], loss: {:.4f}, train_accuracy: {:.4f}, time: {:.2f} s/iter, learning rate: {}".format(total_iters, epoch, total_loss.item(), correct/total, time_cur, exp_lr_scheduler.get_lr()[0]))

        # save model
        if total_iters % args.save_freq == 0:
            msg = 'Saving checkpoint: {}'.format(total_iters)
            _print(msg)
            if multi_gpus:
                net_state_dict = net.module.state_dict()
                margin_state_dict = margin.module.state_dict()
            else:
                net_state_dict = net.state_dict()
                margin_state_dict = margin.state_dict()
            if not os.path.exists(save_dir):
                os.mkdir(save_dir)
            torch.save({
                'iters': total_iters,
                'net_state_dict': net_state_dict},
                os.path.join(save_dir, 'Iter_%06d_net.ckpt' % total_iters))
            torch.save({
                'iters': total_iters,
                'net_state_dict': margin_state_dict},
                os.path.join(save_dir, 'Iter_%06d_margin.ckpt' % total_iters))

print('finishing training')

Train Epoch: 1/18 ...
Train Epoch: 1/18 ...


len(data) 2 torch.Size([32, 3, 112, 112]) torch.Size([32])
len(data) 2 torch.Size([32, 3, 112, 112]) torch.Size([32])
len(data) 2 torch.Size([32, 3, 112, 112]) torch.Size([32])
len(data) 2 torch.Size([32, 3, 112, 112]) torch.Size([32])
len(data) 2 torch.Size([32, 3, 112, 112]) torch.Size([32])
len(data) 2 torch.Size([32, 3, 112, 112]) torch.Size([32])
len(data) 2 torch.Size([32, 3, 112, 112]) torch.Size([32])
len(data) 2 torch.Size([32, 3, 112, 112]) torch.Size([32])
len(data) 2 torch.Size([32, 3, 112, 112]) torch.Size([32])
len(data) 2 torch.Size([32, 3, 112, 112]) torch.Size([32])
len(data) 2 torch.Size([32, 3, 112, 112]) torch.Size([32])
len(data) 2 torch.Size([32, 3, 112, 112]) torch.Size([32])
len(data) 2 torch.Size([32, 3, 112, 112]) torch.Size([32])
len(data) 2 torch.Size([32, 3, 112, 112]) torch.Size([32])
len(data) 2 torch.Size([32, 3, 112, 112]) torch.Size([32])
len(data) 2 torch.Size([32, 3, 112, 112]) torch.Size([32])
len(data) 2 torch.Size([32, 3, 112, 112]) torch.Size([32

Train Epoch: 2/18 ...
Train Epoch: 2/18 ...


len(data) 2 torch.Size([32, 3, 112, 112]) torch.Size([32])
len(data) 2 torch.Size([32, 3, 112, 112]) torch.Size([32])


KeyboardInterrupt: 

# main()

In [26]:
args = EasyDict()

# args.train_root ='../data/faces_webface_112x112/imgs'    #'train image root')
# args.train_file_list= '../data/faces_webface_112x112/train2.lst'    #'train list')
args.train_root =''    #'train image root')
args.train_file_list= '../data/small_vgg/train.lst'    #'train list')
args.lfw_test_root='../data/lfw/'     #'lfw image root')
args.lfw_file_list='/media/sda/lfw/pairs.txt'     #'lfw pair file list')
args.agedb_test_root='/media/sda/AgeDB-30/agedb30_align_112'     #'agedb image root')
args.agedb_file_list='/media/sda/AgeDB-30/agedb_30_pair.txt'     #'agedb pair file list')
args.cfpfp_test_root='/media/sda/CFP-FP/cfp_fp_aligned_112'     #'agedb image root')
args.cfpfp_file_list='/media/sda/CFP-FP/cfp_fp_pair.txt'     #'agedb pair file list')

args.backbone='SERes100_IR'     #'MobileFace, Res50_IR, SERes50_IR, Res100_IR, SERes100_IR, Attention_56, Attention_92')
args.margin_type='ArcFace'     #'ArcFace, CosFace, SphereFace, MultiMargin, Softmax')
args.feature_dim=512     #'feature dimension, 128 or 512')
args.scale_size=32.0     #'scale size')
args.batch_size=32     #'batch size', defalut : 200
args.total_epoch=18     #'total epochs')

args.save_freq=3000     #'save frequency')
args.test_freq=3000     #'test frequency')
args.resume=False     #'resume model')
args.net_path=''     #'resume model')
args.margin_path=''     #'resume model')
args.save_dir='./model'     #'model save dir')
args.model_pre='SERES100_'     #'model prefix')
args.gpus='0,1,2,3'     #'model prefix')


In [6]:
# root = 'D:/data/webface_align_112'
# file_list = 'D:/data/webface_align_train.list'
root = '../data/faces_webface_112x112/imgs'  
file_list = '../data/small_vgg/small_vgg_112x112.lst'
print(os.path.isdir(root), os.path.isfile(file_list))

True False
